In [50]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as patches

class PetalCountUsingHueBasedImageSegmentation:
    img_path = '', img
    
    def __init__(self, image_path, filtered_image_path):
        self.img_path = image_path
        self.filtered_image_path = filtered_image_path
        
    def __load_image(self):
        print(self.img_path)
        self.img = cv2.imread(self.img_path, -1)
        print(self.img)
    
    def __show_loaded_image(self):
        plt.imshow(self.img)
        plt.show()
        
    def __convert_bgr_2_hsi(self):
        normalized_img = np.zeros(self.img.shape).astype(float)
        normalized_img = cv2.normalize(self.img.astype(float), normalized_img, 0, 1, cv2.NORM_MINMAX)
        img_converted = np.zeros_like(self.img, dtype='float')
        for i in range(self.img.shape[0]):
            for j in range(self.img.shape[1]):
                H = 0
                S = 0
                I = 0
                R = float(normalized_img[i][j][2])
                G = float(normalized_img[i][j][1])
                B = float(normalized_img[i][j][0])
                RGB_sum = R + G + B
                RGB_min = min(R, G, B)
                #print("{} ,{} ,{} , {} , {}".format(R,G,B,RGB_min,RGB_sum))
                I = (1/3) * RGB_sum
                if R == G and G == B:
                    H = 0
                    S = 0
                else:
                    S = 1 - ((3*RGB_min)/RGB_sum)
                    if S > 0.9999:
                        S = 1
                    if S < 0.00001:
                        S = 0
                        H = 0
                    else:
                        dividend = (1/2) * ((R-G) + (R-B))
                        divisor = math.sqrt(math.pow((R-G),2) + (R-B)*(G-B))
                        w = dividend/divisor
                        if w > 1:
                            w=1
                        elif w < -1:
                            w = -1                    
                        H = math.acos(w)
                        if B > G:
                            H = (2*math.pi) - H
                img_converted[i][j][0] = H * (180 / math.pi)
                img_converted[i][j][1] = S
                img_converted[i][j][2] = I

        plt.imshow(img_converted)
        print("RGB {},{},{}  , HSI {},{},{}".format(self.img[0][0][2],self.img[0][0][1],self.img[0][0][0],img_converted[0][0][0]
                                                   ,img_converted[0][0][1],img_converted[0][0][2]))
        fig = plt.figure(figsize=(20,20))
        ax = fig.add_subplot(4,4,1)
        plt.imshow(img_converted[0:,0:20])
        ax = fig.add_subplot(4,4,2)
        plt.imshow(img_converted[0:20,0:])
        ax = fig.add_subplot(4,4,3)
        plt.imshow(img_converted[0:,-20:-1])
        ax = fig.add_subplot(4,4,4)
        plt.imshow(img_converted[-20:-1,0:])
        
    def __convert_bgr_2_hsv(self):
        self.HSV_img = cv2.cvtColor(self.img,cv2.COLOR_BGR2HSV)
        plt.imshow(self.HSV_img)
        
    def __get_histogram_of_hsv_img(self):
        plt.hist(self.HSV_img.ravel(),256,[0,256])
        
    def __remove_freq_values(original_image,filter_size,ratio):
        new_image = np.zeros_like(original_image)
        most_freq = []
        mask = np.zeros(original_image.shape[:2],np.uint8)
        mask[0:,0:filter_size] = 1
        hist = cv2.calcHist([original_image],[0],mask,[256],[0,256])
        for i in range(hist.shape[0]):
            if hist[i][0] >= hist[hist.argmax()][0]/ratio:
                most_freq.append(i)

        mask[0:,-filter_size:-1] = 1
        hist = cv2.calcHist([original_image],[0],mask,[256],[0,256])
        mask = np.zeros(original_image.shape[:2],np.uint8)
        for i in range(hist.shape[0]):
            if hist[i][0] >= hist[hist.argmax()][0]/ratio:
                most_freq.append(i)

        mask = np.zeros(original_image.shape[:2],np.uint8)
        mask[0:filter_size,0:] = 1
        hist = cv2.calcHist([original_image],[0],mask,[256],[0,256])
        for i in range(hist.shape[0]):
            if hist[i][0] >= hist[hist.argmax()][0]/ratio:
                most_freq.append(i)

        mask = np.zeros(original_image.shape[:2],np.uint8)
        mask[-filter_size:-1,0:] = 1
        hist = cv2.calcHist([original_image],[0],mask,[256],[0,256])
        for i in range(hist.shape[0]):
            if hist[i][0] >= hist[hist.argmax()][0]/ratio:
                most_freq.append(i)

        print(most_freq)
        for i in range(original_image.shape[0]):
            for j in range(original_image.shape[1]):
                if original_image[i][j][0] not in most_freq:
                    new_image[i][j][0] = original_image[i][j][0]
                    new_image[i][j][1] = original_image[i][j][1]
                    new_image[i][j][2] = original_image[i][j][2]
        if cv2.compare(new_image,original_image,0).all():
            print("Nothing Changed")
        return new_image
    
    
    def __show_hsv_img(self):
        fig = plt.figure(figsize=(20,20))
        ax = fig.add_subplot(4,4,1)
        plt.imshow(self.HSV_img[0:,0:20])
        ax = fig.add_subplot(4,4,2)
        plt.imshow(self.HSV_img[0:20,0:])
        ax = fig.add_subplot(4,4,3)
        plt.imshow(self.HSV_img[0:,-20:-1])
        ax = fig.add_subplot(4,4,4)
        plt.imshow(self.HSV_img[-20:-1,0:])
        
        
    def __normalize_by_removing_freq_values(self):
        self.filterd_HSV_filter = np.zeros_like(self.HSV_img)
        self.filterd_HSV_filter = cv2.normalize(self.HSV_img,filterd_HSV_filter,0,255,cv2.NORM_MINMAX)
        filter_size = 35
        for i in range(4):
            ratio = 4
            mask = np.zeros(self.HSV_img.shape[:2],np.uint8)
            mask[0:,0:filter_size] = 1
            mask[0:,-filter_size:-1] = 1
            mask[0:filter_size,0:] = 1
            mask[-filter_size:-1,0:] = 1
            self.filterd_HSV_filter = __remove_freq_values(self.filterd_HSV_filter, filter_size, ratio)
            fig = plt.figure(figsize=(20,20))
            ax = fig.add_subplot(2,2,1)
            plt.imshow(self.HSV_img)
            ax = fig.add_subplot(2,2,2)
            plt.imshow(self.filterd_HSV_filter)
            filter_size += 5
            ratio += 4
            
            
    def __show_filtered_hsv_filter_image(self):
        self.new_img = cv2.cvtColor(self.filterd_HSV_filter, cv2.COLOR_HSV2BGR)
        self.new_img2 = cv2.medianBlur(self.new_img, 5)
        fig = plt.figure(figsize=(20,20))
        ax = fig.add_subplot(1,3,1)
        plt.imshow(self.img)
        ax = fig.add_subplot(1,3,2)
        plt.imshow(self.new_img)
        ax = fig.add_subplot(1,3,3)
        plt.imshow(self.new_img2)
        
        
    def __convert_2_gray_image_and_save(self):
        self.gray = cv2.cvtColor(self.new_img2, cv2.COLOR_BGR2GRAY)
        if not cv2.imwrite(self.filtered_image_path + 'gray.png', self.gray):
            print("Unable to save in {0} due to internal problem".format(self.filtered_image_path + 'gray.png'))
        plt.imshow(self.gray, cmap='gray')
        plt.show()
        
        
    def __get_threshold_from_gray_image(self):
        ret, self.thresh = cv2.threshold(self.gray, 127, 255, cv2.THRESH_BINARY_INV)
        plt.imshow(self.thresh, cmap='gray')
        plt.show()
        
        
    def __distance(self, x1, y1, x2, y2):
        length = math.sqrt(abs((x1 - x2))**2 + abs((y1 - y2))**2)
        return length
    
    
    def __intersection(self, line1, line2):
        """Finds the intersection of two lines given in Hesse normal form.

        Returns closest integer pixel locations.
        See https://stackoverflow.com/a/383527/5087436
        """
        rho1, theta1 = line1[0]
        rho2, theta2 = line2[0]
        A = np.array([
            [np.cos(theta1), np.sin(theta1)],
            [np.cos(theta2), np.sin(theta2)]
        ])
        b = np.array([[rho1], [rho2]])
        x0, y0 = np.linalg.solve(A, b)
        x0, y0 = int(np.round(x0)), int(np.round(y0))
        return x0, y0
    
    
    
    def __get_petals_count(self):
        
        cnts, hierarchy = cv2.findContours(self.thresh, cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE)
        print("Number of Contours {0}".format(len(cnts)))

        # Create figure and axes
        fig,ax = plt.subplots(1)

        # Display the image
        ax.imshow(self.img)

        count = 1
        petals = 0
        for c in cnts:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio

            #if count == 2:
            (x, y, w, h) = cv2.boundingRect(c)
            if ((h > 300 and w > 300) and (h < self.img.shape[0] and w < self.img.shape[1])):
                print(c)
                contour_area = cv2.contourArea(c)
                rect_area = w * h

                rect = patches.Rectangle((x,y), w, h, linewidth=5, edgecolor='r', facecolor='none')
                ax.add_patch(rect)

                (min_cx, min_cy),radius = cv2.minEnclosingCircle(c)
                center = (int(min_cx),int(min_cy))
                radius = int(radius)
                circle = patches.Circle((min_cx, min_cy), radius=radius, linewidth=5, edgecolor='g',facecolor='none')
                ax.add_patch(circle)
                circle_area = math.pi * radius * radius

                (x, y), (MA, ma), angle = cv2.fitEllipse(c)
                ellipse_area = math.pi * MA * ma

                # determine the most extreme points along the contour
                extLeft = tuple(c[c[:, :, 0].argmin()][0])
                extRight = tuple(c[c[:, :, 0].argmax()][0])
                extTop = tuple(c[c[:, :, 1].argmin()][0])
                extBot = tuple(c[c[:, :, 1].argmax()][0])

                line1 = mlines.Line2D([extLeft[0],extRight[0]], [extLeft[1],extRight[1]])
                line2 = mlines.Line2D([extTop[0],extBot[0]], [extTop[1],extBot[1]])
                line3 = mlines.Line2D([extLeft[0], extTop[0]], [extLeft[1], extTop[1]])
                str_data = "Height {0}, Width {1}, Area {2}, extLeft: {3}, extRight: {4}, extTop: {5}, extBot: {6}, \
                Line1: {7}, Line2: {8}, circle_area: {9}, ellipse_area: {10}, rect_area: {11}" \
                .format(w, h, contour_area, extLeft, extRight, extTop, extBot, line1, line2, \
                        circle_area, ellipse_area, rect_area)
                print(str_data)
                ax.add_line(line1)
                ax.add_line(line2)
                ax.add_line(line3)

                side1 = distance(extLeft[0],extRight[0], extLeft[1], extRight[1])
                side2 = distance(extTop[0],extBot[0], extTop[1], extBot[1])
                side3 = distance(extLeft[0], extTop[0], extLeft[1], extTop[1])
                str_side = "Side 1: {0}, Side 2: {1}, Side 3: {2}".format(side1, side2, side3)
                print(str_side)
                semi_perimeter = (side1 + side2 + side3)//2
                area = math.sqrt(semi_perimeter*(semi_perimeter-side1)*(semi_perimeter-side2)*(semi_perimeter-side3))

                M = cv2.moments(c)
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                center_circle = plt.Circle((cx, cy), 10)
                ax.add_patch(center_circle)

                petals = math.ceil((circle_area *0.85)/area)
                print("Triangle Area: {0}, Contour Area: {1}, petals: {2}".format(area, contour_area, petals))
                
            count = count + 1

        plt.show()
        return petals
    
    @classmethod
    def get_petals_count(self):
        self.__load_image(self)
        self.__show_loaded_image(self)
        self.__convert_bgr_2_hsi(self)
        self.__convert_bgr_2_hsv(self)
        self.__get_histogram_of_hsv_img(self)
        self.__show_hsv_img(self)
        self.__normalize_by_removing_freq_values(self)
        self.__show_filtered_hsv_filter_image(self)
        self.__convert_2_gray_image_and_save(self)
        self.__get_threshold_from_gray_image(self)
        return self.__get_petals_count(self)
        
        
def main_method():
    x = PetalCountUsingHueBasedImageSegmentation(r'../Storage/flowers_for_petals/1/image_06748.jpg',
                                                r'../Storage/preview_filtered_flowers_for_petals/1/')
    petals = x.get_petals_count()
    print("from main method Petals : {0}".format(petals))
    
if __name__ == '__main__':
    print("Start petal count using image segmentation based on hue...")
    main_method()
    print("End petal count using image segmentation based on hue.")

Start petal count using image segmentation based on hue...
('', array([[[ 35,  45,  55],
        [ 36,  46,  56],
        [ 36,  46,  56],
        ...,
        [ 31,  43,  47],
        [ 31,  43,  47],
        [ 31,  43,  47]],

       [[ 32,  42,  52],
        [ 33,  43,  53],
        [ 34,  44,  54],
        ...,
        [ 36,  48,  52],
        [ 36,  48,  52],
        [ 36,  48,  52]],

       [[ 29,  39,  49],
        [ 30,  40,  50],
        [ 31,  41,  51],
        ...,
        [ 39,  51,  55],
        [ 39,  51,  55],
        [ 39,  51,  55]],

       ...,

       [[107, 106, 146],
        [107, 106, 146],
        [105, 106, 144],
        ...,
        [166, 158, 165],
        [168, 157, 165],
        [168, 157, 165]],

       [[119, 118, 158],
        [119, 118, 158],
        [117, 118, 156],
        ...,
        [176, 168, 175],
        [178, 167, 175],
        [178, 167, 175]],

       [[126, 125, 165],
        [125, 124, 164],
        [124, 125, 163],
        ...,
        [1

SystemError: <built-in function imread> returned NULL without setting an error